# Argumentation framework

### Unit testing

In [ ]:
from utils.iccma.index import get_graphs
from utils.test.index import test

TESTING = False

if TESTING:
    graphs = get_graphs([
        ("scc", "small"),
        ("gr", "small"),
        ("st", "small"),
    ])
    # Complete, stable and preferred
    test(
        graphs=graphs,
        solvers=["minisat", "manysat", "maplesat", "glucose"],
        semantics=["CO", "ST", "PR"],
        tasks=["SE", "EE"],
        timeout=100,  # seconds
    )
    # Grounded (must not set the solvers)
    test(
        graphs=graphs,
        solvers=[],
        semantics=["GR"],
        tasks=["SE", "EE"],
        timeout=1,  # seconds
    )
else:
    print("Skipped testing")

### Benchmarking

In [1]:
from utils.iccma.index import get_graphs
from utils.iccma.type import GraphType, Size
from utils.benchmark.index import bench
from utils.benchmark.export import Export
from pathlib import Path
from typing import List

export = Export(Path("results"), "st")
sizes: List[Size] = ["small", "medium", "large"]

for size in sizes:
    bench(
        graphs=get_graphs([("scc", size)]),
        solvers=["minisat", "manysat", "maplesat", "glucose"],
        semantics=["ST"],
        tasks=["EE"],
        timeout=5 * 60,  # seconds
        export=export,
    )

Initialised _ReproducibleRandom with seed 164854496363
100%          
100%          
100%          


### Graph generation

In [ ]:
from utils.gen.stable import gen as gen_stable
from utils.gen.grounded import gen as gen_grounded
# from utils.gen.scc import gen as gen_scc

g_stable = gen_stable(500, 100, 500, 100, 500, 100, 500, .6)
g_stable.save("stable")

g_grounded = gen_grounded(500, .3)
g_grounded.save("grounded")

# g_scc = gen_scc(500, 500, .6, .3, .4)
# g_scc.save("scc")

In [ ]:
import networkx as nx
from utils.graph import Graph

nx_graph = nx.Graph()
nx_graph.add_nodes_from("hello")
nx_graph.add_edge("h", "e")

graph = Graph.from_networkx(nx_graph)
graph.save("networkx")

graph2 = Graph.from_networkx(nx.Graph(nx.binomial_tree(4)))
graph2.save("bitree")


### TODO

In [1]:
from utils.iccma.index import get_graphs
from utils.graph import Graph


iccma_graphs = get_graphs([("st", "small")])
graph = Graph.from_iccma_graph(iccma_graphs[0])
graph.get_non_cred_arg("ST")

EE-ST
C:\Users\quent\AppData\Local\Temp\tmppstpn213\get_arg.tgf
[['a59', 'a9', 'a69', 'a91', 'a2', 'a131', 'a111', 'a31', 'a57', 'a27', 'a96', 'a45', 'a67', 'a6', 'a14', 'a5', 'a44', 'a25', 'a65', 'a128', 'a30', 'a52', 'a53', 'a99', 'a75', 'a7', 'a3', 'a79', 'a56', 'a10', 'a16', 'a48', 'a83', 'a21', 'a37', 'a101', 'a54', 'a103', 'a35', 'a85', 'a11', 'a88', 'a20', 'a50', 'a107'], ['a25', 'a128', 'a67', 'a75', 'a31', 'a20', 'a50', 'a2', 'a30', 'a101', 'a54', 'a45', 'a96', 'a3', 'a56', 'a37', 'a48', 'a65', 'a69', 'a59', 'a52', 'a28', 'a6', 'a44', 'a83', 'a5', 'a21', 'a10', 'a85', 'a57', 'a103', 'a91', 'a9', 'a107', 'a111', 'a14', 'a79', 'a11', 'a35', 'a53', 'a88', 'a7', 'a16', 'a136'], ['a28', 'a10', 'a107', 'a56', 'a31', 'a9', 'a16', 'a44', 'a54', 'a69', 'a96', 'a101', 'a67', 'a53', 'a25', 'a83', 'a65', 'a35', 'a11', 'a48', 'a57', 'a5', 'a21', 'a30', 'a3', 'a20', 'a7', 'a45', 'a50', 'a37', 'a13', 'a91', 'a103', 'a75', 'a85', 'a128', 'a52', 'a136', 'a79', 'a2', 'a6', 'a59', 'a62'], ['a67'

'a77'

In [ ]:
# bench(
#     solvers=["manysat", "minisat"],
#     semantics="CO",
#     tasks="SE",
#     graphs=[(a, gen_grounded(a, .3)) for a in range(0, 10)],
#     abs="max arg",
#     ord="time",
# )

# SUPPOSED TO:
# Générer une image d'un graphe pour manysat et un autre pour minisat
# (les deux courbes sur le même graphique)
# avec la sémanntique CO et la tâche SE
# en abscisses: a
# en ordonnées: le temps d'exécution pour résoudre gen_grounded(a, .3)
# du coup il y aurait 10 points par courbe

# AUTRE EXEMPLE
# bench(
#     solvers=["manysat", "minisat"],
#     semantics="CO",
#     tasks="SE",
#     graphs=[(count_arguments(file), file) for file in graph_files],
#     abs="max arg",
#     ord="time",
# )